# **Introduction:**

This file serves to design and test a custom implementation of an Adaptive Neuro-Fuzzy Inference System (ANFIS) within Keras. It is hoped that this custom function is able to be used like a standard Keras model, and will be trained and evaluated against the ANN designed previously. 

This ANFIS takes the load history, the distance to the task, and the total distance travelled thus far and performs inference about the suitability of a given robot for a task at hand. 

**Date Created:** 13/01/2025

**Date Modified:** 14/01/2025

# **Import Packages:** 

This section imports all the necessary packages for the ANFIS implementation. 

In [15]:
# import packages:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Sequential
from tensorflow.keras.optimizers import Adam

# **Layer Function & Class Definitions:**

Need to define the membership function to be used, as well as the custom layers of an ANFIS for implementation within Keras.

In [5]:
# define a function for triangular membership functions:
def triangular_mf(x, a, b, c):
    return tf.maximum(0.0, tf.minimum((x - a) / (b - a), (c - x) / (c - b)))

# custom fuzzification layer:
class FuzzificationLayer(layers.Layer):

    # constructor to initialize objects:
    def __init__(self, num_inputs, num_mfs):
        super(FuzzificationLayer, self).__init__()  # initialize in the same manner as the parent
        self.num_inputs = num_inputs                # assign the number of inputs
        self.num_mfs = num_mfs                      # assign the number of membership functions

        # create learnable mf parameters
        self.mf_params = self.add_weight(           
            shape = (num_inputs, num_mfs, 3),       # shape being num_inputs rows, num_mfs columns, and a depth of 3 for the 3 parameters in a triangular membership function
            initializer = 'random_uniform',         # randomly, uniformly initialize the weights
            trainable = True,                       # set to be trainable
        )

    # forward pass through layer given by:
    def call(self, inputs):
        mfs = []            # initialize empty list
        for i in range(self.num_inputs):        # for each input x_i
            mfs.append(                         # add the output to the mf list
                tf.stack(
                    # each input i across all instances is inputted, along with the parameters for the j-th membership function for the i-th input
                    [triangular_mf(inputs[:,i], *self.mf_params[i, j]) for j in range(self.num_mfs)], axis = 1
                )
            )
        return tf.concat(mfs, axis = 1) # return to user
    
# custom firing strength layer:
class FiringStrengthLayer(layers.Layer):
    def call(self, inputs):
        rules = tf.reduce_prod(inputs, axis = 1, keepdims = True)
        return rules
    
# custom normalization layer:
class NormalizationLayer(layers.Layer):
    def call(self, inputs):
        return inputs / tf.reduce_sum(inputs, axis = 1, keepdims = True)
    
# custom consequent layer:
class ConsequentLayer(layers.Layer):

    # constructor to initialize objects:
    def __init__(self, num_rules, num_inputs):
        super(ConsequentLayer, self).__init__()     # initialize in the same manner as the parent
        self.params = self.add_weight(              # takagi-sugeno consequent parameters
            shape = (num_rules, num_inputs + 1),
            initializer = 'random_uniform',
            trainable = True
        )

    # forward pass through layer given by:
    def call(self, inputs):
        # add a bias term to the inputs:
        bias = tf.ones((inputs.shape[0], 1))
        extended_inputs = tf.concat([inputs, bias], axis = 1)
        return tf.matmul(extended_inputs, self.params, transpose_b=True)
    
# custom output layer:
class OutputLayer(layers.Layer):
    def call(self, inputs):
        return tf.reduce_sum(inputs, axis=1, keepdims=True)

# **Define Testing Parameters:**

This section defines the testing parameters, such as the number of inputs, the number of membership functions, and the expected number of rules. This section also defines a function for creating and compiling Keras models using the custom layers.

Testing Parameters:

In [6]:
# define parameters:
num_inputs = 3
num_mfs = 3
num_rules = num_mfs ** num_inputs

Model Generation Function:

In [10]:
# function to make models:
def make_model(num_inputs, num_mfs, num_rules, rate):

    # instantiate model:
    model = Sequential()

    # add fuzzification layer:
    model.add(FuzzificationLayer(num_inputs = num_inputs, num_mfs = num_mfs))

    # add firing strength layer:
    model.add(FiringStrengthLayer())

    # add normalization layer:
    model.add(NormalizationLayer())

    # add rule consequent layer:
    model.add(ConsequentLayer(num_rules = num_rules, num_inputs = num_inputs))

    # add output layer:
    model.add(OutputLayer())
    
    # return model:
    return model

# **Define Hybrid Training Function:**

It is important to note that training an ANFIS involves the use of a hybrid training algorithm usually, where the antecedent parameters (membership function parameters) are updated using back propagation through gradient descent, whereas the consequent parameters (rule consequent parameters) are updated using least-square optimization. This isn't natively supported within Keras, so it must be defined as follows:

In [11]:
# define hybrid training function:

def hybrid_train(model, train_dataset, num_epochs, rate):
    # define an optimizer:
    optimizer = Adam(learning_rate = rate)

    # for every epoch:
    for epoch in range(num_epochs):
        # for every x,y batch in the dataset:
        for x_batch, y_batch in train_dataset:
            # first must use backpropagation for antecedents:
            with tf.GradientTape() as tape:
                outputs = model(x_batch, training = True)                   # get forward pass output
                loss = tf.reduce_mean(tf.square(y_batch - outputs))         # calculate MSE 

            # gradient descent:
            antecedent_variables = model.layers[0].trainable_variables      # access trainable variables of fuzzification layer
            grads = tape.gradient(loss, antecedent_variables)               # get the gradient
            optimizer.apply_gradients(zip(grads, antecedent_variables))     # apply the gradient to the variables

            # now we can do least squares optimization for consequents:
            firing_strengths = model.layers[1](model.layers[0](x_batch))    # firing strength layer output
            normalized_strengths = model.layers[2](firing_strengths)        # normalized firing strength layer output
            extended_inputs = tf.concat([normalized_strengths, tf.ones_like(normalized_strengths)], axis = 1)

            # solve for consequents using least-squares:
            consequent_weights = tf.linalg.lstsq(extended_inputs, y_batch, l2_regularizer=1e-3)
            model.layers[3].params.assign(tf.transpose(consequent_weights)) # update consequent parameters
        
        # logging the loss for monitoring
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.numpy():.4f}")
    
    return model

# **Loading of Training Data:**

Need to load the data that had been generated using the FIS, such that it may be used to train the ANFIS model:

In [29]:
# load data from the CSV:
df = pd.read_csv('V3_Data.csv')

# extract the X and Y components of the dataframe:
x_data = df.drop(['Suitability'], axis = 1)
y_data = df['Suitability']

# need to split the data into training, validation, and testing:
x_train, x_temp, y_train, y_temp = train_test_split(x_data, y_data, test_size = 0.2)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size = 0.5)

# get split results:
print(f"there are {x_train.shape[0]} training examples")
print(f"there are {x_val.shape[0]} validation examples")
print(f"there are {x_test.shape[0]} testing examples")

there are 8000 training examples
there are 1000 validation examples
there are 1000 testing examples


Now need to perform normalization to improve model performance:

In [30]:
# normalize the training data:
x_min = x_train.min(axis = 0)
x_max = x_train.max(axis = 0)
x_train = (x_train - x_min) / (x_max - x_min)

# apply the normalization to the validation and testing data:
x_val = (x_val - x_min) / (x_max - x_min)
x_test = (x_test - x_min) / (x_max - x_min)

Convert to TensorFlow tensors:

In [31]:
# tensor conversion:
x_train_tensor = tf.convert_to_tensor(x_train, dtype = tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype = tf.float32)
x_val_tensor = tf.convert_to_tensor(x_val, dtype = tf.float32)
y_val_tensor = tf.convert_to_tensor(y_val, dtype = tf.float32)
x_test_tensor = tf.convert_to_tensor(x_test, dtype = tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype = tf.float32)

# batch the data:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_tensor, y_train_tensor)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val_tensor, y_val_tensor)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_tensor, y_test_tensor)).batch(batch_size)